In [12]:
import pandas as pd

In [15]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"
# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [102 kB]
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:13 http://archive.u

In [17]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2023-01-06 00:44:13--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.33MB/s    in 0.2s    

2023-01-06 00:44:14 (6.33 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [22]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M17-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [23]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [24]:
from pyspark.sql.functions import to_date

In [28]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show(5)

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows



In [30]:
from google.colab import files
uploaded = files.upload()

Saving vine_table.csv to vine_table.csv


In [31]:
import pandas as pd
import io
df = pd.read_csv(io.BytesIO(uploaded['vine_table.csv']))

In [32]:
df.head()

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R1NBG94582SJE2,5.0,0.0,0.0,N,Y
1,R273DCA6Y0H9V7,5.0,0.0,0.0,N,Y
2,RQVOXO7WUOFK6,2.0,1.0,1.0,N,Y
3,R1KWKSF21PO6HO,5.0,0.0,0.0,N,Y
4,R38H3UO1J190GI,5.0,1.0,1.0,N,Y


In [33]:
count_df = df.loc[df["total_votes"] >= 20]
count_df

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
66,R3FJ319XA6ZAUQ,2.0,21.0,25.0,N,Y
124,R6HRF25HUMIIE,5.0,24.0,24.0,N,Y
173,R2CTAK3APOFKZU,5.0,37.0,38.0,N,N
217,R1WQZB0CAEPQBU,2.0,85.0,128.0,N,N
221,RHZDSA48HVRVK,5.0,49.0,86.0,N,N
...,...,...,...,...,...,...
1801963,R29H40OYUHM7JS,1.0,61.0,62.0,N,N
1801968,R1QRN4ICSNTW30,3.0,32.0,32.0,N,N
1801969,R2II0GWVA7GL4J,5.0,24.0,25.0,N,N
1801972,RYBK7XWLUMO67,5.0,43.0,46.0,N,N


In [34]:
helpful_df = count_df.loc[(count_df["helpful_votes"] / count_df["total_votes"]) >= .5]
helpful_df

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
66,R3FJ319XA6ZAUQ,2.0,21.0,25.0,N,Y
124,R6HRF25HUMIIE,5.0,24.0,24.0,N,Y
173,R2CTAK3APOFKZU,5.0,37.0,38.0,N,N
217,R1WQZB0CAEPQBU,2.0,85.0,128.0,N,N
221,RHZDSA48HVRVK,5.0,49.0,86.0,N,N
...,...,...,...,...,...,...
1801963,R29H40OYUHM7JS,1.0,61.0,62.0,N,N
1801968,R1QRN4ICSNTW30,3.0,32.0,32.0,N,N
1801969,R2II0GWVA7GL4J,5.0,24.0,25.0,N,N
1801972,RYBK7XWLUMO67,5.0,43.0,46.0,N,N


In [35]:
paid_df = helpful_df.loc[helpful_df["vine"] == "Y"]
paid_df

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
4370,R37L6YZDX0P3J9,1.0,28.0,35.0,Y,N
5627,R2JT4XRQO2KASL,5.0,38.0,41.0,Y,N
32659,R10XRAOBFFSTL2,5.0,44.0,47.0,Y,N
34040,R1X8F3R1K1SF5,2.0,30.0,31.0,Y,N
57741,R326QV66CKBB43,4.0,127.0,139.0,Y,N
...,...,...,...,...,...,...
1680504,R1FR2EEJXQ0J9U,2.0,33.0,37.0,Y,N
1680796,R2L8OMPMDIC5QK,4.0,81.0,84.0,Y,N
1680813,R1RY93JL20IW5B,4.0,153.0,160.0,Y,N
1682745,R1RR0YI6ODJ2V5,5.0,22.0,23.0,Y,N


In [36]:
unpaid_df = helpful_df.loc[helpful_df["vine"] == "N"]
unpaid_df

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
66,R3FJ319XA6ZAUQ,2.0,21.0,25.0,N,Y
124,R6HRF25HUMIIE,5.0,24.0,24.0,N,Y
173,R2CTAK3APOFKZU,5.0,37.0,38.0,N,N
217,R1WQZB0CAEPQBU,2.0,85.0,128.0,N,N
221,RHZDSA48HVRVK,5.0,49.0,86.0,N,N
...,...,...,...,...,...,...
1801963,R29H40OYUHM7JS,1.0,61.0,62.0,N,N
1801968,R1QRN4ICSNTW30,3.0,32.0,32.0,N,N
1801969,R2II0GWVA7GL4J,5.0,24.0,25.0,N,N
1801972,RYBK7XWLUMO67,5.0,43.0,46.0,N,N


In [37]:
paid_total = paid_df["review_id"].count()
paid_total

607

In [38]:
unpaid_total = unpaid_df["review_id"].count()
unpaid_total

50522

In [39]:
paid_5star = paid_df.loc[(paid_df["star_rating"] == 5.0)]
paid_5star.head(5)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
5627,R2JT4XRQO2KASL,5.0,38.0,41.0,Y,N
32659,R10XRAOBFFSTL2,5.0,44.0,47.0,Y,N
73187,RGXLFTR31J561,5.0,29.0,34.0,Y,N
105646,R8O8KM73GO190,5.0,21.0,28.0,Y,N
113535,RPU1GURA1NOT0,5.0,33.0,35.0,Y,N


In [40]:
unpaid_5star = unpaid_df.loc[(unpaid_df["star_rating"] == 5.0)]
unpaid_5star.head(5)

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
124,R6HRF25HUMIIE,5.0,24.0,24.0,N,Y
173,R2CTAK3APOFKZU,5.0,37.0,38.0,N,N
221,RHZDSA48HVRVK,5.0,49.0,86.0,N,N
404,R1WZSWWOTN58OP,5.0,73.0,77.0,N,Y
594,R29THVJFO35FZA,5.0,48.0,50.0,N,Y


In [41]:
paid_5star_count = paid_5star["star_rating"].count()
paid_5star_count

257

In [42]:
unpaid_5star_count = unpaid_5star["star_rating"].count()
unpaid_5star_count

25220

In [43]:
paid_5staar_prcnt = paid_5star_count / paid_total * 100
paid_5staar_prcnt

42.33937397034596

In [44]:
unpaid_5stsr_prcnt = unpaid_5star_count / unpaid_total * 100
unpaid_5stsr_prcnt

49.91884723486798